## Protokoli za istovetenje

In [ ]:
import flask
# pip install flask
# pip install --user flask


## App z Flask (Python)

## VP library

In [ ]:
# Import necessary modules
import os
import string

from cryptography.exceptions import InvalidKey
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt


def generate_response_hmac_256(key: bytes, challenge: str) -> str:
    """Izračuna odziv za podan poziv"""

    # Pri implementaciji si pomagajate z Javansko različico:
    #  String generateResponseHMAC256(byte[] key, String challenge)
    return "TODO"


def generate_challenge() -> str:
    """Ustvari naključno 6-mestno številko"""
    rnd = os.urandom(4)
    number = int.from_bytes(rnd, byteorder="big")
    number = number & 0x7FFFFFFF  # nastavi MSB na 0 (da bo pozitivno)
    number = number % 1000000  # ostanek pri deljenju z 10^6 da 6-mestno število
    # vrnemo 6 mestni niz: če je število manj-mestno, ga podložimo z ničlami
    return "{0:06d}".format(number)


def hash_password(password: str) -> tuple[bytes, bytes]:
    """Iz podanega gesla izračuna zgoščeno vrednost s funkcijo scrypt.

    Vrne par (sol, zgostitev)"""
    salt = os.urandom(16)

    kdf = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1)
    hashed = kdf.derive(password.encode("utf8"))

    return salt, hashed


def verify_password(attempt: str, salt: bytes, hashed: bytes) -> bool:
    """Vzame niz poskus, bajte sol in bajte, ki predstavljajo zgoščeno vrednost
    pravega gesla in vrne True ntk. je poskus gesla pravilen. Sicer vrne False."""
    kdf = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1)
    try:
        kdf.verify(attempt.encode("utf8"), hashed)
        return True
    except InvalidKey:
        return False

## App

In [ ]:
from flask import Flask, request

app = Flask(__name__)

# Baza uporabnikov
# - Kljuc predstavlja uporabnika, vrednosti pa sta par (sol, zgoščeno geslo)
# - Podan je primer za Ano in za geslo "vp"
_salt, _hashed = vp_library.hash_password("vp")
users = {
    'ana': (_salt, _hashed),
}


@app.route('/', methods=['GET'])
def login():
    return '''
        <!doctype html>
        <h1>Log in</h1>
        <form action="/validate" method="post">
            Username: <input type="text" name="username"><br>
            Password: <input type="password" name="password"><br>
            <br>
            <input type="submit" value="Login">
        </form>
        <p>[
            <a href="/">Log-in</a> |
            <a href= "/register">Register</a>
        ]'''


@app.route('/register', methods=['GET'])
def register():
    return '''
        <!doctype html>
        <h1>Register</h1>
        <form action="/register" method="post">
            Username: <input type="text" name="username"><br>
            Password: <input type="password" name="password"><br>
            <br>
            <input type="submit" value="Register">
        </form>
        <p>[
            <a href="/">Log-in</a> |
            <a href= "/register">Register</a>
        ]'''


@app.route('/register', methods=['POST'])
def register_save():
    username = request.form['username']
    password = request.form['password']

    if username in users:
        return "Username already taken."

    salt, hashed = vp_library.hash_password(password)
    users[username] = salt, hashed

    return f'''
        <!doctype html>
        <h1>Register</h1>
        <p>Welcome, {username}, the registration was successful.
        <p>[
            <a href="/">Log-in</a> |
            <a href= "/register">Register</a>
        ]'''



@app.route('/validate', methods=['POST'])
def validate_user_pass():
    username = request.form['username']
    password = request.form['password']

    msg = f"Welcome, {username}, the passsword was correct."
    if not (username in users and vp_library.verify_password(password, *users[username])):
        msg = "Invalid username/password"

    return f'''
        <!doctype html>
        <h1>Login status</h1>
        <p>{msg}
        <p>[
            <a href="/">Log-in</a> |
            <a href= "/register">Register</a>
        ]'''


if __name__ == '__main__':
    app.run(debug=True)


## HOTP:

In [ ]:
import hmac, hashlib, struct, base64

def hotp(secret, counter, digits=6):
    key = base64.b32decode(secret)
    msg = struct.pack(">Q", counter)
    hmac_hash = hmac.new(key, msg, hashlib.sha1).digest()

    offset = hmac_hash[-1] & 0x0F
    binary = ((hmac_hash[offset] & 0x7f) << 24 |
              (hmac_hash[offset + 1] & 0xff) << 16 |
              (hmac_hash[offset + 2] & 0xff) << 8 |
              (hmac_hash[offset + 3] & 0xff))

    otp = binary % (10 ** digits)
    return str(otp).zfill(digits)


secret = "JBSWY3DPEHPK3PXP"  # base32 koda
print(hotp(secret, 1))


## TOTP:

In [ ]:
import time
def totp(secret, interval=30, digits=6):
    counter = int(time.time()) // interval
    return hotp(secret, counter, digits)

print(totp("JBSWY3DPEHPK3PXP"))
